### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [2]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [3]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn
tfidfvect = TfidfVectorizer()

In [4]:
# en el atributo `data` accedemos al texto
newsgroups_train.data[0]

'I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.'

In [5]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [6]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'cantidad de documentos: {X_train.shape[0]}')
print(f'tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
cantidad de documentos: 11314
tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [7]:
# una vez fiteado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [8]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [9]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [10]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [11]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [12]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [13]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.70930477, 0.67474953, ..., 0.        , 0.        ,
       0.        ])

In [14]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([ 4811,  6635,  4253, ...,  1534, 10055,  4750], dtype=int64)

In [15]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [16]:
# el documento original pertenece a la clase:
newsgroups_test.target_names[y_train[idx]]

'talk.politics.misc'

In [17]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_test.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


### Modelo de clasificación Naïve Bayes

In [18]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [19]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [20]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

### Consigna del desafío 1

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares.


##### Ejercicio 1

In [21]:
import random

num_documents = 5
random_documents_indices = np.random.choice(X_train.shape[0], num_documents, replace=False)
random_documents = X_train[random_documents_indices]

In [22]:
similarities = cosine_similarity(random_documents, X_train)

In [23]:
for i, random_index in enumerate(random_documents_indices):
    sorted_indices = np.argsort(similarities[i])[::-1]
    top_similar_documents_indices = sorted_indices[1:6] 

    print(f"\nRandom Document {i + 1}: {newsgroups_test.target_names[y_train[i]]}")
    print(newsgroups_train.data[random_index])
    print("\nTop 5 Most Similar Documents:")
    
    for j, similar_index in enumerate(top_similar_documents_indices):
        print(f"\nSimilar Document {j + 1} (Similarity: {similarities[i][similar_index]:.4f}): {newsgroups_test.target_names[y_train[i]]}")
        print(newsgroups_train.data[similar_index])


Random Document 1: rec.autos



WHO THINKS THE ASTROS ARE GOING PLACES???
THEY'RE CURRENTLY FIRST PLACE.
THEY'RE 5-4, 5-1 ON THE ROAD! 


Top 5 Most Similar Documents:

Similar Document 1 (Similarity: 0.2612): rec.autos
                                      ^^^^^^
No argument at all with Murphy.  He scared the hell out of me when he came in
last year.  On the other hand, the club though enough of Boever to put him into
an awful lot of games (he may have led the league in appearances - he did at
least at some point).  He seemed to be a very viable setup guy - but I guess
that's not considered that crucial by the club.  I can just remember two years
ago so well, though...
...

I'm not that concerned.  Those guys have been relatively consistent over the
years and they have no good reasons to decline (no injuries, not old, ...).
I expect them to come through just fine.  It's those guys that have not
been consistently good that are the worrisome part, even if they are coming
through right 

##### Ejercicio 2

In [24]:
from sklearn.metrics import f1_score, classification_report
from sklearn.model_selection import GridSearchCV

clf = MultinomialNB()
clf.fit(X_train, y_train)

X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

f1_score(y_test, y_pred, average='macro')

0.5854345727938506

In [25]:
parameters = {
    'alpha': [0.1, 1.0, 10.0],
    'class_prior': [None, [0.2, 0.8], [0.5, 0.5]],
    'fit_prior': [True, False],
    'force_alpha': [True, False]
}

In [26]:
grid_search_mnb = GridSearchCV(clf, parameters, scoring='f1_macro', cv=5)
grid_search_mnb.fit(X_test, y_test)

C:\Users\cbureu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
120 fits failed out of a total of 180.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
120 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\cbureu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\cbureu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\loca

GridSearchCV(cv=5, estimator=MultinomialNB(),
             param_grid={'alpha': [0.1, 1.0, 10.0],
                         'class_prior': [None, [0.2, 0.8], [0.5, 0.5]],
                         'fit_prior': [True, False],
                         'force_alpha': [True, False]},
             scoring='f1_macro')

In [27]:
best_mnb_model = grid_search_mnb.best_estimator_

In [28]:
y_pred_mnb = best_mnb_model.predict(X_test)
f1_mnb = f1_score(y_test, y_pred_mnb, average='macro')
print("Multinomial Naive Bayes F1-Score:", f1_mnb)
print(classification_report(y_test, y_pred_mnb))


Multinomial Naive Bayes F1-Score: 0.9282882531222951
              precision    recall  f1-score   support

           0       0.56      0.92      0.69       319
           1       0.93      0.97      0.95       389
           2       0.96      0.91      0.93       394
           3       0.93      0.96      0.95       392
           4       0.99      0.94      0.96       385
           5       0.99      0.96      0.98       395
           6       0.97      0.93      0.95       390
           7       0.99      0.91      0.95       396
           8       0.99      0.96      0.97       398
           9       0.99      0.94      0.97       397
          10       0.99      0.96      0.98       399
          11       0.98      0.94      0.96       396
          12       0.99      0.94      0.96       393
          13       0.99      0.95      0.97       396
          14       0.99      0.93      0.96       394
          15       0.78      0.96      0.86       398
          16       0.90     

In [29]:
cnb = ComplementNB()
cnb.fit(X_train, y_train)

X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  cnb.predict(X_test)

f1_score(y_test, y_pred, average='macro')

0.692953349950875

In [30]:
grid_search_cnb = GridSearchCV(cnb, parameters, scoring='f1_macro', cv=5)
grid_search_cnb.fit(X_test, y_test)

C:\Users\cbureu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
120 fits failed out of a total of 180.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
120 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\cbureu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\cbureu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\loca

GridSearchCV(cv=5, estimator=ComplementNB(),
             param_grid={'alpha': [0.1, 1.0, 10.0],
                         'class_prior': [None, [0.2, 0.8], [0.5, 0.5]],
                         'fit_prior': [True, False],
                         'force_alpha': [True, False]},
             scoring='f1_macro')

In [31]:
best_cnb_model = grid_search_cnb.best_estimator_

In [32]:
y_pred_cnb = best_cnb_model.predict(X_test)
f1_cnb = f1_score(y_test, y_pred_cnb, average='macro')
print("Multinomial Naive Bayes F1-Score:", f1_cnb)
print(classification_report(y_test, y_pred_cnb))

Multinomial Naive Bayes F1-Score: 0.9312048760884123
              precision    recall  f1-score   support

           0       0.56      0.92      0.70       319
           1       0.94      0.95      0.94       389
           2       0.95      0.89      0.92       394
           3       0.93      0.95      0.94       392
           4       0.98      0.93      0.95       385
           5       0.96      0.96      0.96       395
           6       0.95      0.94      0.94       390
           7       0.99      0.91      0.95       396
           8       0.99      0.95      0.97       398
           9       0.99      0.93      0.96       397
          10       0.98      0.97      0.98       399
          11       0.98      0.95      0.96       396
          12       0.97      0.94      0.95       393
          13       0.99      0.95      0.97       396
          14       0.98      0.94      0.96       394
          15       0.86      0.95      0.91       398
          16       0.90     

##### Ejercicio 3

In [48]:
term_document_matrix = X_train.T
print(term_document_matrix.shape)

(101631, 11314)


In [49]:
num_words = 5
random_word_indices = np.random.choice(term_document_matrix.shape[0], num_words, replace=False)

In [50]:
for word_index in random_word_indices:
    word = idx2word[word_index]
    word_vector = term_document_matrix[word_index].reshape(1, -1)
    similarities = cosine_similarity(word_vector, term_document_matrix)

    # Obtener los índices de las 5 palabras más similares
    sorted_indices = np.argsort(similarities[0])[::-1]
    top_similar_words_indices = sorted_indices[1:6]  # Excluir la palabra en sí misma

    print(f"\nPalabra: {word}")
    print("5 Palabras Más Similares:")
    
    for similar_index in top_similar_words_indices:
        similar_word = idx2word[similar_index]
        similarity_score = similarities[0][similar_index]
        print(f"\nPalabra Similar: {similar_word} (Similitud: {similarity_score:.4f})")



Palabra: 5vznkjz
5 Palabras Más Similares:

Palabra Similar: fpkjz (Similitud: 1.0000)

Palabra Similar: yl1u (Similitud: 1.0000)

Palabra Similar: yl6a (Similitud: 1.0000)

Palabra Similar: yl7 (Similitud: 1.0000)

Palabra Similar: 5tl71x (Similitud: 1.0000)

Palabra: montesque
5 Palabras Más Similares:

Palabra Similar: russia (Similitud: 0.5279)

Palabra Similar: democracy (Similitud: 0.3513)

Palabra Similar: slaves (Similitud: 0.3473)

Palabra Similar: deserve (Similitud: 0.2602)

Palabra Similar: nation (Similitud: 0.2601)

Palabra: xtal
5 Palabras Más Similares:

Palabra Similar: synthasized (Similitud: 0.9234)

Palabra Similar: indestructable (Similitud: 0.9234)

Palabra Similar: synthaszied (Similitud: 0.9234)

Palabra Similar: bander (Similitud: 0.9234)

Palabra Similar: 2watt (Similitud: 0.7593)

Palabra: vedah
5 Palabras Más Similares:

Palabra Similar: vedah (Similitud: 1.0000)

Palabra Similar: upanisads (Similitud: 1.0000)

Palabra Similar: entanglement (Similitud: 1.00